# Importing needed modules

In [ ]:
%matplotlib inline

import numpy as np
import sep
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib import rcParams
from matplotlib.patches import Ellipse


rcParams['figure.figsize'] = [10., 8.]

# Naming variable with image name

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
hdu_list.info()

In [ ]:
image_data = fits.getdata(fname)
image_data = image_data.byteswap().newbyteorder()
print(type(image_data))
print(image_data.shape)

# Show the image

In [ ]:
m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();


# Histogram the Flux of image

In [ ]:
histogram = plt.hist(image_data.flatten(), bins = 'auto')

# Delineating between foreground and background

In [ ]:
bkg = sep.Background(image_data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = np.array(bkg)
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

In [ ]:
data_sub = image_data - bkg

# Finding number of objects in the image

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
len(objects)

# Outlining the found objects

In [ ]:
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))